In [0]:
from pyspark.sql import functions as f
import json
from urllib.request import urlopen

bloburl = dbutils.widgets.get('bloburl')
isFileBased = dbutils.widgets.get('isFileBased').lower() == 'true'

try:
    response = urlopen(bloburl)
except Exception as e:
    print(e)

data = ""
for line in response:
    data+=line.decode()
try:
    config = json.loads(data)
except Exception as e:
    print(e)
print(config)

# setting azure storage account access using spark.conf
spark.conf.set(
    "fs.azure.account.key."+config["storageAccountName"]+".dfs.core.windows.net",
    dbutils.secrets.get(scope=config["scope"],key=config["key"])) 

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1360241541083845> in <module> 
 3 from urllib . request import urlopen
 4 
 ----> 5 bloburl = dbutils . widgets . get ( 'bloburl' ) 
 6 isFileBased = dbutils . widgets . get ( 'isFileBased' ) . lower ( ) == 'true' 
 7 

 /databricks/python_shell/dbruntime/WidgetHandlerImpl.py in get (self, name) 
 40 : return : Current value of the widget or default value
 41 """
 ---> 42 return self . _notebookArguments . getArgument ( name , self . _entry_point . getCurrentBindings ( ) ) 
 43 
 44 def getArgument ( self , name , defaultValue = None ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o277.getArgument.
: com.databricks.dbutils_v1.InputWidgetNotDefined: No input widget named bloburl is defined
	at com.databricks.backend.daemon.driver.NotebookArguments.checkExists(NotebookArguments.scala:72)
	at com.databricks.backend.daemon.driver.NotebookArguments.getArgument(NotebookArguments.scala:258)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [0]:
SalesLT = {}
sourceDir = config["sourceDir"]

#if file based then data will be taken from file based directory, else from bronze zone source directory
if isFileBased:
    sourceDir = config ['fileBasedDir']
    
for table in config['tablenames']:
    try:
        SalesLT[table] = spark.read.format("parquet").option("header","true").load("abfss://"+config["container"]+"@"+config["storageAccountName"]+".dfs.core.windows.net/"+sourceDir + "/" + table)
    except Exception as e:
        print(e)

In [0]:
from pyspark.sql import functions as f

print("BEFORE CASTING")
SalesLT["SalesOrderDetail"].printSchema()
SalesLT["SalesOrderDetail"] = SalesLT["SalesOrderDetail"].withColumn("SalesOrderID",f.col("SalesOrderID").cast("bigint"))
print("AFTER CASTING")
SalesLT["SalesOrderDetail"].printSchema()

In [0]:
SalesLT_ColumnTypes = {}

for table in config['tablenames']:
    #storing column types of each table to handle null
    SalesLT_ColumnTypes[table] = dict(SalesLT[table].dtypes)

In [0]:
def handleNull(DF,cols,colTypes):
    """
        Args:
            DF(Spark.DataFrame): DataFrame Object on which analysis should be performed
            cols(list): list of column names that has to be null handled
            colTypes(dictionary): dictionary of column name and its data types
        Returns:
            Spark.DataFrame: Null handled DataFrame Object
    """
    for col in cols:
        if colTypes[col] == "string":
            DF = DF.na.fill({col:config["replaceNullValue"]["string"]})
        elif colTypes[col] == "timestamp":
            DF = DF.na.fill({col:config["replaceNullValue"]['timestamp']})
        elif colTypes[col] == "boolean":
            DF = DF.na.fill({col:config["replaceNullValue"]['boolean']})
        else:
            DF = DF.na.fill({col:config["replaceNullValue"]['numeric']})
    return DF

In [0]:
def dropColumns(tablename,df,cols):
    """
        Args:
            tablename: Name of the DataFrame Object Table
            DF(Spark.DataFrame): DataFrame Object on which analysis should be performed
            cols(list): list of column names that has to be dropped
        Returns:
            Spark.DataFrame: DataFrame Object without columns in 'cols'
    """
    for col in cols:
        df=df.drop(col)
    return df

In [0]:
for table in config['tablenames']:
    SalesLT[table] = handleNull(SalesLT[table],config["nullHandling"][table],SalesLT_ColumnTypes[table])

In [0]:
for table in config['tablenames']:
    SalesLT[table]= dropColumns(table,SalesLT[table],config["dropColumns"][table])

In [0]:
print("BEFORE RENAMING")
SalesLT["Address"].printSchema()
SalesLT["Address"]=SalesLT["Address"].withColumnRenamed("AddressLine1","AddressL1")
SalesLT["Address"]=SalesLT["Address"].withColumnRenamed("AddressLine2","AddressL2")
print("AFTER RENAMING")
SalesLT["Address"].printSchema()


In [0]:
SalesLT["Product"].select(f.col("Size"),
          f.col("Size").cast("int").isNotNull().alias("Value")
  ).show()

In [0]:
SalesLT["Product"]=SalesLT["Product"].withColumn("Size",
                   f.when((SalesLT["Product"].Size>=38) & (SalesLT["Product"].Size<=42), 'S')
                   .when((SalesLT["Product"].Size>=44) & (SalesLT["Product"].Size<=48), 'M')
                   .when((SalesLT["Product"].Size>=50) & (SalesLT["Product"].Size<=54), 'L')
                   .when((SalesLT["Product"].Size>=56) & (SalesLT["Product"].Size<=62), 'XL')
                   .otherwise(SalesLT["Product"].Size))

SalesLT["Product"].show()

In [0]:
ProductWithSizeM = SalesLT["Product"].filter("Size ='M'")
ProductWithSizeM.show()

In [0]:
for table in config['tablenames']:
        SalesLT[table].write.format('delta').mode("overwrite").saveAsTable(table+"Table")

In [0]:
%sql
describe extended CustomerAddressTable

In [0]:
%sql
INSERT into CustomerAddressTable values(12345,567,"Main Office",CURRENT_TIMESTAMP)

In [0]:
%sql
UPDATE CustomerAddressTable
SET AddressType = 'Airport'
WHERE CustomerID = 12345;

In [0]:
%sql
DELETE FROM CustomerAddressTable
WHERE CustomerID = 12345;

In [0]:
for table in config['tablenames']:
        SalesLT[table] = spark.sql("select * from "+table+"Table") 

In [0]:
#Writing All Tables into Silverzone
for table in config['tablenames']:
    try:
        SalesLT[table].write.format("parquet").option("header","true").mode("overwrite").save("abfss://"+config["container"]+"@"+config["storageAccountName"]+".dfs.core.windows.net/"+config["sinkDir"]+"/"+table)
    except Exception as e:
        print(e)
    

In [0]:

import numpy
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.functions import isnan, when, count, col

def dataprofile(data_all_df):
    data_cols = data_all_df.columns
    data_df = data_all_df.select(data_cols)
    columns2Bprofiled = data_df.columns
    
    global schema_name, table_name
    if not 'schema_name' in globals():
        schema_name = 'schema_name'
    if not 'table_name' in globals():
        table_name = 'table_name' 
    dprof_df = pd.DataFrame({'column_names':data_df.columns,\
                             'data_types':[x[1] for x in data_df.dtypes]}) 
    dprof_df = dprof_df[['column_names', 'data_types']]
    
    # ======================
    
    num_rows = data_df.count()
    dprof_df['num_rows'] = num_rows
    
    # ======================    
    
    # number of rows with nulls and nans   
    df_nacounts = data_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data_df.columns \
                                  if data_df.select(c).dtypes[0][1]!='timestamp' and data_df.select(c).dtypes[0][1]!='binary' and data_df.select(c).dtypes[0][1]!='boolean']).toPandas().transpose()
    df_nacounts = df_nacounts.reset_index()  
    df_nacounts.columns = ['column_names','num_null']
    dprof_df = pd.merge(dprof_df, df_nacounts, on = ['column_names'], how = 'left')
    
    # ========================
    
    # number of rows with white spaces (one or more space) or blanks
    num_spaces = [data_df.where(F.col(c).rlike('^\\s+$')).count() for c in data_df.columns]
    dprof_df['num_spaces'] = num_spaces
    num_blank = [data_df.where(F.col(c)=='').count() for c in data_df.columns]
    dprof_df['num_blank'] = num_blank
    
    # =========================
    
    # using the in built describe() function 
    desc_df = data_df.describe().toPandas().transpose()
    desc_df.columns = ['count', 'mean', 'stddev', 'min', 'max']
    desc_df = desc_df.iloc[1:,:]  
    desc_df = desc_df.reset_index()  
    desc_df.columns.values[0] = 'column_names'  
    desc_df = desc_df[['column_names','count', 'mean', 'stddev']] 
    dprof_df = pd.merge(dprof_df, desc_df , on = ['column_names'], how = 'left')
    
    # ===========================================    
    
    df_counts = dprof_df[['column_names']]
    dprof_df = pd.merge(dprof_df, df_counts , on = ['column_names'], how = 'left')  
    
    # ==========================================
    
    # number of distinct values in each column
    dprof_df['num_distinct'] = [data_df.select(x).distinct().count() for x in columns2Bprofiled]
    
    # ============================================
    
    # most frequently occuring value in a column and its count
    dprof_df['most_freq_valwcount'] = [data_df.groupBy(x).count().sort("count",ascending=False).limit(1).\
                                       toPandas().iloc[0].values.tolist() for x in columns2Bprofiled]
    dprof_df['most_freq_value'] = [x[0] for x in dprof_df['most_freq_valwcount']]
    dprof_df['most_freq_value_count'] = [x[1] for x in dprof_df['most_freq_valwcount']]
    dprof_df = dprof_df.drop(['most_freq_valwcount'],axis=1)
    
    # ============================================
    
    # least frequently occuring value in a column and its count
    dprof_df['least_freq_valwcount'] = [data_df.groupBy(x).count().sort("count",ascending=True).limit(1).\
                                        toPandas().iloc[0].values.tolist() for x in columns2Bprofiled]
    dprof_df['least_freq_value'] = [x[0] for x in dprof_df['least_freq_valwcount']]
    dprof_df['least_freq_value_count'] = [x[1] for x in dprof_df['least_freq_valwcount']]
    dprof_df = dprof_df.drop(['least_freq_valwcount'],axis=1)

    return dprof_df

In [0]:
from datetime import datetime

now = datetime.now()
date_time = now.strftime("%m-%d-%Y, %H:%M:%S")
#With each run, new profile statistics will be created for each table in a new folder using date_time as folder name

if isFileBased:
    date_time = date_time + " - File"

for table in config['tablenames']:
    result = dataprofile(SalesLT[table]) 
    resultDF = spark.createDataFrame(result.astype(str))
    resultDF.repartition(1).write.option('header',True).mode('overwrite').format('csv').save("abfss://"+config["container"]+"@"+config["storageAccountName"]+".dfs.core.windows.net/"+"Profile Statistics"+"/"+table+"/"+date_time)

